# Simple Error Model for Twinkles

This notebook will calculate a simple error model for the Twinkles data as a function of the seeing and depth.

### Requirements

You will need the DESC `Monitor` and its dependencies.

You will also need the star_cache.db from  in your `Monitor/data` directory.

Finally, you need the correct opsim database for your Twinkles run. For run 1.1 this is kraken_1042 and for run 3 and beyond this is minion_1016. Both are available [here](https://www.lsst.org/scientists/simulations/opsim/opsim-v335-benchmark-surveys).

In [ ]:
import os
import desc.monitor
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

### Get differences between true and observed fluxes

We first want to get all the differences for the run between the true and observed fluxes and store them in a Pandas Dataframe.

In [ ]:
star_db_name = 'twinkles_run1.1.db'

In [ ]:
truth_dbConn = desc.monitor.truthDBInterface(database=star_db_name, driver='sqlite')
dbConn_twinkles = desc.monitor.dbInterface(database='DESC_Twinkles_Level_2',
                                           host='127.0.0.1', port='3307',
                                           driver='mysql')

In [ ]:
worker = desc.monitor.Monitor(dbConn_twinkles, truth_dbConn=truth_dbConn)

In [ ]:
depth_curve = worker.measure_depth_curve()

In [ ]:
seeing_curve = worker.measure_seeing_curve()

In [ ]:
worker.calc_flux_diff(with_depth_curve=depth_curve, with_seeing_curve=seeing_curve)

In [ ]:
max_s = np.max(worker.flux_stats['seeing'])
min_s = np.min(worker.flux_stats['seeing'])
delta_s = (max_s - min_s)//10.
max_d = np.max(worker.flux_stats['depth'])
min_d = np.min(worker.flux_stats['depth'])
delta_d = (max_d - min_d)//10.

In [ ]:
d_grid, s_grid = np.meshgrid(np.arange(min_d, max_d+(delta_d/2.), delta_d), np.arange(min_s, max_s+(delta_s/2.), delta_s))
p_val = np.zeros(np.shape(d_grid))
for depth in d_grid:
    print depth
    depth_grid_vals = worker.flux_stats[((worker.flux_stats['depth'] > depth) & (worker.flux_stats['depth'] < (depth + delta_d)))]
    for seeing in s_grid:
        grid_vals = depth_grid_vals[((depth_grid_vals['seeing'] > seeing) & (depth_grid_vals['seeing'] < seeing + delta_s))]
        p_val[depth, seeing] = 2

In [ ]:
plt.scatter(worker.flux_stats['depth'], worker.flux_stats['seeing'], c=worker.flux_stats['bias'])
plt.colorbar()

In [ ]:
plt.scatter(worker.flux_stats['depth'], worker.flux_stats['seeing'], c=worker.flux_stats['sigma'])
plt.colorbar()

In [ ]:
worker.calc_statistics()